### Tutorial: Minimum Observations Given a Budget

In this simplest example, let's say you have a budget of 60 observations on a spectrograph, and you want to spend them on a single target. How do you best allocate your budget to maximize the Fisher Information on the radial velocity semi-amplitude, K? (In other words, how do you minimize the uncertainty on K?)

In this simplest example, let's say you have a target consisting of planetary system around a single cool, active dwarf. For this example, we'll use AU Mic. Let's say that, given an observing strategy of once per night, we want to know how many observations we would need to achieve a threshold uncertainty on the radial velocity semi-amplitude, K, of 10%. For AU Mic, with a K of 580 cm/s (Klein+ 2021), that means we want the minimum observations needed to reach an uncertainty of 58 cm/s. 

We'll start with the single planet case and then generalize to include both AU Mic b and c. 

In [1]:
import numpy as np 
import scipy
print(np.__version__)
print(scipy.__version__)
import tqdm
import pandas as pd
import random
import exoplanet
import astropy 
import pymc3
import pymc3_ext
import celerite2
from numpy.linalg import inv, det, solve, cond
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
#from jax import random

1.22.3
1.7.3


In [2]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)

path = '/Users/chrislam/Desktop/scheduling/'

In [5]:
from gaspery import calculate_fi, strategies, utils


1.22.3
1.7.3
1.22.3
1.7.3


In [7]:
sigma_ks = []
c = 1 # observing cadence of once per day

for n_obs in range(100)[4:]:
    
    # instantiate Strategy class in order to build time series of observations
    strategy = Strategy(n_obs, c, start, offs=[], dropout=0.)
    
    # build strategy aka time series of observations
    strat = strategies.gappy(strategy)

    # build covariance matrix, characterized by a correlated noise model of the stellar signal
    sigma = np.diag(sigma_wn_rv * np.ones(len(strat)))

    # populate arguments for Fisher Info calculator
    args = np.array(strat), sigma**2, jnp.array(theta, dtype=float)
    
    # calculate FI
    fim = calculate_fi.clam_jax_fim(*args).block_until_ready()
    
    # invert FI matrix
    inv_fim = inv(fim)
    
    # top left element of matrix corresponds with RV semi-amplitude, K
    sigma_k = np.sqrt(inv_fim)[0][0]
    
    sigma_ks.append(sigma_k)
    

AttributeError: module 'gaspery.strategies' has no attribute 'gappy'

In [6]:
calculate_fi.model_jax()

TypeError: model_jax() missing 4 required positional arguments: 't', 'K', 'P', and 'T0'